# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
#nltk.download('wordnet')
#nltk.download('punkt')
#nltk.download('stopwords')

In [2]:
# import libraries
import re
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
import numpy as np

import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV

from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline

import datetime
from pytz import timezone

In [3]:
# load data from database

database_filename ='data/DisasterResponse.db'
engine = create_engine('sqlite:///' + database_filename)

df = pd.read_sql("SELECT * FROM disaster_response", engine)

X = df['message']
Y = df.iloc[:,4:]
category_list = list(df.columns[4:])

df.head()

#print(Y.shape)
#df.columns.get_loc("direct_report")


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", ' ', text.lower())

    # Tokenize text
    tokens = word_tokenize(text)
    
    # Remove Stopwords
  #  tokens = [t for t in tokens if t not in stopwords.words("english")]
    
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens



def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)
    
    
    
#def load_data(filepath):

In [5]:
# test Tokenize function
text = X[0]
tokens = tokenize(text)
print(tokens)

['weather', 'update', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))])



### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# perform train test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.2, random_state=101)

# train classifier
pipeline.fit(X_train,Y_train) 


C:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabula...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.


The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.

The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

The F-beta score can be interpreted as a weighted harmonic mean of the precision and recall, where an F-beta score reaches its best value at 1 and worst score at 0.

The F-beta score weights recall more than precision by a factor of beta. beta == 1.0 means recall and precision are equally important.

In [8]:
# predict on test data

y_pred = pipeline.predict(X_test)
print(type(y_pred))

# display Accuracy Report

for i in range(len(category_list)):

        print("Message Category:", category_list[i],"\n", classification_report(Y_test.iloc[:, i].values, y_pred[:, i]))
        print('Accuracy of  Category: %25s: %.2f' %(category_list[i], accuracy_score(Y_test.iloc[:, i].values, y_pred[:,i])))

<class 'numpy.ndarray'>
Message Category: related 
               precision    recall  f1-score   support

           0       0.60      0.35      0.44      1251
           1       0.81      0.93      0.87      3949
           2       0.50      0.36      0.42        36

    accuracy                           0.78      5236
   macro avg       0.64      0.55      0.58      5236
weighted avg       0.76      0.78      0.76      5236

Accuracy of  Category:                   related: 0.78
Message Category: request 
               precision    recall  f1-score   support

           0       0.88      0.98      0.93      4297
           1       0.81      0.41      0.54       939

    accuracy                           0.88      5236
   macro avg       0.85      0.69      0.74      5236
weighted avg       0.87      0.88      0.86      5236

Accuracy of  Category:                   request: 0.88
Message Category: offer 
               precision    recall  f1-score   support

           0       1.

C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


 other_infrastructure 
               precision    recall  f1-score   support

           0       0.95      1.00      0.98      4990
           1       0.17      0.00      0.01       246

    accuracy                           0.95      5236
   macro avg       0.56      0.50      0.49      5236
weighted avg       0.92      0.95      0.93      5236

Accuracy of  Category:      other_infrastructure: 0.95
Message Category: weather_related 
               precision    recall  f1-score   support

           0       0.85      0.96      0.90      3841
           1       0.83      0.54      0.66      1395

    accuracy                           0.85      5236
   macro avg       0.84      0.75      0.78      5236
weighted avg       0.85      0.85      0.84      5236

Accuracy of  Category:           weather_related: 0.85
Message Category: floods 
               precision    recall  f1-score   support

           0       0.94      1.00      0.97      4811
           1       0.87      0.27      0

### 6. Improve your model
Use grid search to find better parameters. 

:n_estimators = number of trees in the foreset 
:.max_features = max number of features considered for splitting a node
:.max_depth = max number of levels in each decision tree
:.min_samples_split = min number of data points placed in a node before the node is split
:.min_samples_leaf = min number of data points allowed in a leaf node
:bootstrap = method for sampling data points (with or without replacement)


In [9]:
pipeline.get_params()

{'memory': None,
 'steps': [('vectorizer',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=None, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                                                          max_features='auto',
                                    

In [11]:
tz = timezone('EST')
start_time  = datetime.datetime.now(tz)
print('Model Execution start time...', start_time) 



parameters = {'clf__estimator__n_estimators': [50],
               'clf__estimator__min_samples_split': [2, 4, 6],
               'clf__estimator__max_depth': [2,5,10],
               'clf__estimator__min_samples_leaf': [3,4,5]
               

                 

                 }

pipeline_grid = GridSearchCV(pipeline, param_grid= parameters,cv= 3, n_jobs=4)
print("Model Built....")

# train classifier
print("Training classifier....")
pipeline_grid.fit(X_train,Y_train) 

end_time = datetime.datetime.now(tz)
print('Model Execution End Time',end_time) 
print('Model Execution Duration(in Secs) ',(end_time-start_time).seconds) 

#print(grid_search.best_params_)

Model Execution start time... 2019-11-02 07:58:30.163013-05:00
Model Built....
Training classifier....
Model Execution End Time 2019-11-02 08:02:26.951774-05:00
Model Execution Duration(in Secs)  236


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
# predict on test data using Improved Model

y_pred = pipeline_grid.predict(X_test)
print(type(y_pred))

# display Accuracy Report

for i in range(len(category_list)):

        print("Message Category:", category_list[i],"\n", classification_report(Y_test.iloc[:, i].values, y_pred[:, i]))
        print('Accuracy of  Category: %25s: %.2f' %(category_list[i], accuracy_score(Y_test.iloc[:, i].values, y_pred[:,i])))

<class 'numpy.ndarray'>
Message Category: related 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00      1251
           1       0.75      1.00      0.86      3949
           2       0.00      0.00      0.00        36

    accuracy                           0.75      5236
   macro avg       0.25      0.33      0.29      5236
weighted avg       0.57      0.75      0.65      5236

Accuracy of  Category:                   related: 0.75
Message Category: request 
               precision    recall  f1-score   support

           0       0.82      1.00      0.90      4297
           1       0.00      0.00      0.00       939

    accuracy                           0.82      5236
   macro avg       0.41      0.50      0.45      5236
weighted avg       0.67      0.82      0.74      5236

Accuracy of  Category:                   request: 0.82
Message Category: offer 
               precision    recall  f1-score   support

           0       1.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [16]:
# save the model to disk
filename = 'models/classifier.pkl'
pickle.dump(pipeline_grid, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.